# 1. Imports des bibliothèques nécessaires

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import ipywidgets as widgets
import requests
from google.colab import files
import datetime
import json

# 2. Téléchargement et traitement des données MITRE ATT&CK

In [2]:
# URL des données JSON de MITRE ATT&CK
mitre_json_url = "https://github.com/mitre/cti/raw/master/enterprise-attack/enterprise-attack.json"

# Fonction pour télécharger les données JSON
def download_mitre_json(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Erreur dans la récupération des données JSON.")
        return None

# Charger les données JSON
mitre_data = download_mitre_json(mitre_json_url)

# Filtrer pour obtenir seulement les techniques d'attaque
def extract_techniques(data):
    techniques = [obj for obj in data['objects'] if obj['type'] == 'attack-pattern']
    return techniques

# Extraire les techniques d'attaque
techniques_data = extract_techniques(mitre_data)

# 3. Définir les risques par secteur

In [3]:
# Données des risques par secteur
sectors = {
    "Énergie": ["Attaques sur les infrastructures critiques", "Sabotage des systèmes industriels", "Ransomware"],
    "Finance": ["Vol de données financières", "Fraude transactionnelle", "Phishing"],
    "Industrie": ["Espionnage industriel", "Sabotage de la production", "Vol de propriété intellectuelle"],
    "Commerce": ["Vol de données clients", "Fraude transactionnelle", "Attaques sur les systèmes de point de vente"]
}

# Définir les mots-clés pour filtrer les techniques par secteur
sector_keywords = {
    "Énergie": ["ransomware", "phishing", "infrastructure"],
    "Finance": ["credential", "data", "financial"],
    "Industrie": ["supply", "destruction", "industrial"],
    "Commerce": ["valid", "proxy", "denial"]
}

# Filtrer les techniques pour chaque secteur
def filter_techniques(sector_keywords, techniques_data):
    filtered_techniques = {}
    for sector, keywords in sector_keywords.items():
        filtered_techniques[sector] = [tech for tech in techniques_data if any(keyword in tech['name'].lower() for keyword in keywords)]
    return filtered_techniques

mitre_attacks_dynamic = filter_techniques(sector_keywords, techniques_data)

# 4. Interface Utilisateur pour les données de l'entreprise

In [4]:
# Widgets pour l'entrée des données de l'entreprise
sector_dropdown = widgets.Dropdown(
    options=list(sectors.keys()),
    description='Secteur:',
    disabled=False,
)

revenue_input = widgets.FloatText(
    value=0,
    description='CA (en M€):',
    disabled=False,
)

telework_checkbox = widgets.Checkbox(
    value=False,
    description='Télétravail:',
    disabled=False,
)

# 5. Calcul des coûts et primes d'assurance

In [5]:
# Fonction pour calculer les coûts potentiels d'une cyberattaque
def calculate_costs(sector, revenue, telework):
    base_cost = revenue * 0.05  # Coût de base estimé à 5% du CA
    if telework:
        base_cost *= 1.2  # Augmenter le coût de 20% si télétravail
    return base_cost

# Fonction pour calculer la prime d'assurance
def calculate_insurance_premium(sector, revenue, telework, selected_measures):
    base_premium = calculate_costs(sector, revenue, telework) * 0.05  # Base premium is 5% of estimated cost
    discount_factor = 1.0
    for measure in selected_measures:
        if measure in security_measures:
            discount_factor *= security_measures[measure]
    final_premium = base_premium * discount_factor  # Prime finale avec deux décimales
    return final_premium

# 6. Interface Utilisateur pour les mesures de sécurité

In [6]:
# Widgets pour la sélection des techniques de sécurité mises en place
# Les chiffres ci-dessous doivent être ajustés selon les données réelles et les conseils d'experts
security_measures = {
    "MFA (Multi-Factor Authentication)": 0.95,
    "Encryption": 0.90,
    "Regular Security Audits": 0.85,
    "Employee Training": 0.80,
    "Advanced Firewall": 0.75
}

security_measures_checkboxes = [widgets.Checkbox(value=False, description=measure) for measure in security_measures]

# 7. Génération et téléchargement du rapport

In [7]:
# Fonction mise à jour pour générer un rapport en HTML enrichi avec MITRE ATT&CK, simulation d'attaque, et prime d'assurance
def generate_report(sector, revenue, telework, scenario, selected_security_measures):
    risks = sectors.get(sector, [])
    techniques = mitre_attacks_dynamic.get(sector, [])
    telework_status = "Oui" if telework else "Non"
    estimated_cost = calculate_costs(sector, revenue, telework)
    report_date = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    scenario_description = attack_scenarios[scenario]
    insurance_premium = calculate_insurance_premium(sector, revenue, telework, selected_security_measures)

    report_html = f"""
    <html>
    <head><title>Rapport pour le secteur {sector}</title></head>
    <body style="background-color: #000000; color: #ffffff;">
    <h1>Rapport pour le secteur: {sector}</h1>
    <h2>Données de l'entreprise :</h2>
    <ul>
        <li>Chiffre d'affaires : {revenue} M€</li>
        <li>Télétravail : {telework_status}</li>
        <li>Coût estimé d'une cyberattaque : {estimated_cost:.2f} M€</li>
        <li>Date du rapport : {report_date}</li>
    </ul>
    <h2>Risques identifiés :</h2>
    <ul>
    """
    for risk in risks:
        report_html += f"<li>{risk}</li>"
    report_html += "</ul><h2>Techniques MITRE ATT&CK :</h2><ul>"
    for tech in techniques:
        tech_id = tech.get('external_references', [{}])[0].get('external_id', 'N/A')
        description = tech.get('description', 'Aucune description disponible')
        report_html += f"<li><b>Technique {tech_id}:</b> {tech['name']} - {description}</li>"
    report_html += f"""
    </ul>
    <h2>Simulation d'attaque :</h2>
    <ul>
        <li><b>Scénario sélectionné :</b> {scenario}</li>
        <li><b>Impact potentiel :</b> {scenario_description}</li>
    </ul>
    <h2>Calculateur de Primes d'Assurance :</h2>
    <ul>
        <li><b>Prime d'assurance estimée :</b> {insurance_premium:.2f} €</li>
        <li><b>Techniques de sécurité mises en place :</b> {', '.join(selected_security_measures)}</li>
    </ul>
    </body></html>
    """
    return report_html

# Fonction mise à jour pour télécharger le rapport
def download_report(sector, revenue, telework, scenario, selected_security_measures):
    report_html = generate_report(sector, revenue, telework, scenario, selected_security_measures)
    with open(f'rapport_{sector}.html', 'w') as f:
        f.write(report_html)
    files.download(f'rapport_{sector}.html')

# Bouton pour télécharger le rapport mis à jour
download_button = widgets.Button(
    description='Télécharger le Rapport',
    disabled=False,
    button_style='success',
    tooltip='Cliquez pour télécharger le rapport',
    icon='download',
    layout=widgets.Layout(width='30%', margin='0 auto')
)

def on_download_button_clicked(b):
    sector = sector_dropdown.value
    revenue = revenue_input.value
    telework = telework_checkbox.value
    scenario = scenario_dropdown.value
    selected_security_measures = [checkbox.description for checkbox in security_measures_checkboxes if checkbox.value]
    download_report(sector, revenue, telework, scenario, selected_security_measures)

download_button.on_click(on_download_button_clicked)

# 8. Interface Utilisateur pour la simulation d'attaques

In [8]:
# Scénarios d'attaque
attack_scenarios = {
    "Ransomware": "Impact majeur sur les opérations, données chiffrées, demande de rançon",
    "Phishing": "Vol d'identifiants, accès non autorisé aux systèmes",
    "DDoS": "Interruption des services, perte de revenus",
    "Intrusion": "Accès non autorisé aux données sensibles, vol de données",
    "Malware": "Infection des systèmes, corruption des données"
}

# Widgets pour la sélection du scénario d'attaque
scenario_dropdown = widgets.Dropdown(
    options=list(attack_scenarios.keys()),
    description='Scénario d\'attaque:',
    disabled=False,
)

# 9. Ajout de l'option de téléchargement des données brutes

In [9]:
# Fonction pour générer et télécharger les données brutes
def download_raw_data():
    raw_data = {
        "sectors": sectors,
        "mitre_attacks_dynamic": {sector: [tech['name'] for tech in techniques] for sector, techniques in mitre_attacks_dynamic.items()}
    }
    with open('raw_data.json', 'w') as f:
        json.dump(raw_data, f)
    files.download('raw_data.json')

# Bouton pour télécharger les données brutes
download_raw_data_button = widgets.Button(
    description='Télécharger les Données Brutes',
    disabled=False,
    button_style='warning',
    tooltip='Cliquez pour télécharger les données brutes',
    icon='download',
    layout=widgets.Layout(width='30%', margin='0 auto')
)

def on_download_raw_data_button_clicked(b):
    download_raw_data()

download_raw_data_button.on_click(on_download_raw_data_button_clicked)

# 10. Page d'accueil et lancement de l'application

In [10]:
def display_home_page():
    home_page_html = """
    <html>
    <head>
        <title>Application d'Évaluation des Risques Cyber</title>
        <style>
            body {
                background-color: #000000;
                font-family: Arial, sans-serif;
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
                margin: 0;
                color: #ffffff;
            }
            .container {
                text-align: center;
                padding: 50px;
                background-color: #333333;
                border-radius: 10px;
                box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            }
            h1 {
                color: #ffffff;
            }
            p {
                font-size: 18px;
                color: #ffffff;
                margin-top: 20px;
            }
            ul {
                text-align: left;
                display: inline-block;
                color: #ffffff;
            }
            button {
                margin-top: 20px;
                padding: 10px 20px;
                font-size: 16px;
                color: white;
                background-color: #007bff;
                border: none;
                border-radius: 5px;
                cursor: pointer;
            }
            button:hover {
                background-color: #0056b3;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Bienvenue dans l'Application d'Évaluation des Risques Cyber</h1>
            <p>Cette application permet d'évaluer les risques cybernétiques pour différents secteurs d'activité, de simuler des attaques, et de générer des rapports détaillés.</p>
            <p>Fonctionnalités :</p>
            <ul>
                <li>Évaluation des risques par secteur</li>
                <li>Estimation des coûts potentiels d'une cyberattaque</li>
                <li>Simulation d'attaques pour comprendre l'impact potentiel</li>
                <li>Génération de rapports détaillés incluant les techniques MITRE ATT&CK</li>
                <li>Calculateur de primes d'assurance en fonction des mesures de sécurité mises en place</li>
                <li>Téléchargement des données brutes utilisées dans l'évaluation</li>
            </ul>
        </div>
    </body>
    </html>
    """
    display(HTML(home_page_html))

display_home_page()

def run_app():
    display(sector_dropdown)
    display(revenue_input)
    display(telework_checkbox)
    display(scenario_dropdown)
    for checkbox in security_measures_checkboxes:
        display(checkbox)
    display(download_button)
    display(download_raw_data_button)

run_app()

Dropdown(description='Secteur:', options=('Énergie', 'Finance', 'Industrie', 'Commerce'), value='Énergie')

FloatText(value=0.0, description='CA (en M€):')

Checkbox(value=False, description='Télétravail:')

Dropdown(description="Scénario d'attaque:", options=('Ransomware', 'Phishing', 'DDoS', 'Intrusion', 'Malware')…

Checkbox(value=False, description='MFA (Multi-Factor Authentication)')

...

Checkbox(value=False, description='Advanced Firewall')

Button(button_style='success', description='Télécharger le Rapport', icon='download', layout=Layout(margin='0 …

Button(button_style='warning', description='Télécharger les Données Brutes', icon='download', layout=Layout(ma…

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>